In [1]:
import os
import cv2
import pickle
import h5py
import numpy as np

In [2]:
def preprocess_frame(frame, target_size=(200, 100)):
    """
    Preprocesses a frame for gaze prediction.
    Args:
    - frame: The input image frame (assumed to be in BGR format as per OpenCV standard)
    - target_size: The target size to which the frame should be resized (width, height)

    Returns:
    - Preprocessed frame
    """
    # Check if image is loaded correctly
    if frame is None:
        raise ValueError("Invalid input frame")

    # Convert to grayscale
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to get a binary image
    _, binary_image = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError("No contours found in the frame")

    # Find the largest contour based on area
    largest_contour = max(contours, key=cv2.contourArea)

    # Get the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Crop the image using the bounding box
    cropped_image = frame[y:y+h, x:x+w]

    # Resize the cropped image to the target size
    resized_image = cv2.resize(cropped_image, target_size)
 
    # Convert to float and normalize
    preprocessed_image = resized_image.astype(np.float32) / 255.0

    return preprocessed_image

In [3]:
def get_h5_file_path(subdir):

    # Extract the parent directory of the 'subdir' to get the user's data directory
    user_data_dir = os.path.dirname(subdir)
    
    # Construct the path to the 'Calibration' directory
    calibration_dir = os.path.join(user_data_dir, 'Calibration')
    
    # Assuming there's only one .h5 file per user in the 'Calibration' directory
    for file in os.listdir(calibration_dir):
        if file.endswith('.h5'):
            if 'screenSize' in file:
                return os.path.join(calibration_dir, file)
    
    # If no .h5 file is found, return None or raise an error
    return None

In [4]:

def ask_to_continue(current_dir, processed_dirs_count, subdir_limit=5):
    if processed_dirs_count % subdir_limit == 0:
        answer = input(f"Processed {processed_dirs_count} directories up to {current_dir}. Continue? [y/n]: ")
        if answer.lower() != 'y':
            return False
    return True


In [5]:
import os
import cv2
import h5py
import pickle

def read_h5(h5_file_path):
    with h5py.File(h5_file_path, 'r') as h5_file:
        h5_data = {key: h5_file[key][:] for key in h5_file.keys()}
    return h5_data

def normalize_annotations(annotation, width_pixel, height_pixel):
    normalized_x = float(annotation[0]) / width_pixel
    normalized_y = float(annotation[1]) / height_pixel
    return [normalized_x, normalized_y]

def append_to_pickle(data, filename):
    try:
        with open(filename, 'rb') as file:
            existing_data = pickle.load(file)
    except FileNotFoundError:
        existing_data = {'X': [], 'Y': []}

    existing_data['X'].extend(data['X'])
    existing_data['Y'].extend(data['Y'])

    with open(filename, 'wb') as file:
        pickle.dump(existing_data, file)

def process_images_and_annotations(base_path, subdir_limit=1):
    X, Y = [], []
    batch_number = 0
    last_processed_dir = ""
    processed_dirs_count = 0

    try:
        with open('progress_log.txt', 'r') as file:
            batch_number, last_processed_dir = file.read().strip().split('\n')
            batch_number = int(batch_number)
    except FileNotFoundError:
        print("No progress log found. Starting from the beginning.")

    for subdir, dirs, files in os.walk(base_path):
        if processed_dirs_count >= subdir_limit:
            break  # Stop processing once the subdir limit is reached

        if subdir <= last_processed_dir:
            continue

        print(f"Processing directory {subdir}")

        if not any(file.endswith('.jpg') for file in files):
            continue

        processed_dirs_count += 1
        
        user = os.path.basename(os.path.dirname(subdir))
        h5_file_path = os.path.join(base_path, user, 'Calibration', 'screenSize.h5')
        
        if not os.path.exists(h5_file_path):
            print(f"No .h5 file found for user {user}. Skipping.")
            continue

        h5_data = read_h5(h5_file_path)
        width_pixel, height_pixel = h5_data['width_pixel'][0,0], h5_data['height_pixel'][0,0]

        annotation_file = os.path.join(subdir, 'annotation.txt')
        if not os.path.exists(annotation_file):
            print(f"No annotation file found for directory {subdir}. Skipping.")
            continue

        with open(annotation_file, 'r') as ann_file:
            annotations = [line.strip().split() for line in ann_file]

        for file in sorted(files):
            if file.endswith('.jpg'):
                image_path = os.path.join(subdir, file)
                image_index = sorted(files).index(file)
                if image_index < len(annotations):
                    annotation = normalize_annotations(annotations[image_index][24:26], width_pixel, height_pixel)
                    image = cv2.imread(image_path)
                    preprocessed_image = preprocess_frame(image)
                    X.append(preprocessed_image)
                    Y.append(annotation)

        append_to_pickle({'X': X, 'Y': Y}, f'batches/data_batch_{batch_number}.pkl')
        del X, Y
        X, Y = [], []
        batch_number += 1
        with open('progress_log.txt', 'w') as file:
            file.write(f"{batch_number}\n{subdir}")

    return X, Y, batch_number


In [7]:
# Load and preprocess the images and annotations
images, gaze_data, batch_number = process_images_and_annotations('../MPIIGaze/Data/Original', subdir_limit=50)

Processing directory ../MPIIGaze/Data/Original\p13\day09
Processing directory ../MPIIGaze/Data/Original\p13\day10
Processing directory ../MPIIGaze/Data/Original\p13\day11
Processing directory ../MPIIGaze/Data/Original\p13\day12
Processing directory ../MPIIGaze/Data/Original\p14
Processing directory ../MPIIGaze/Data/Original\p14\Calibration
Processing directory ../MPIIGaze/Data/Original\p14\day01
Processing directory ../MPIIGaze/Data/Original\p14\day02
Processing directory ../MPIIGaze/Data/Original\p14\day03
Processing directory ../MPIIGaze/Data/Original\p14\day04
Processing directory ../MPIIGaze/Data/Original\p14\day05
Processing directory ../MPIIGaze/Data/Original\p14\day06
Processing directory ../MPIIGaze/Data/Original\p14\day07


In [11]:
#clear memory
import gc
del images
del gaze_data
gc.collect()

21

In [ ]:
#plot predicted vs actual on test data on a canvas using opencv 

import cv2
import numpy as np 
predictions = model.predict(X_test)
screen_width, screen_height = 2650, 1440
canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
cv2.namedWindow('Gaze Tracking on Canvas', cv2.WINDOW_NORMAL)
cv2.setWindowProperty('Gaze Tracking on Canvas', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# plot the first 10 images one by one
for i in range(0,25):
    
    # get the predicted x,y coordinates
    x, y = predictions[i][0] * screen_width, predictions[i][1] * screen_height

    # lock the preds 
    x = min(max(x, 0), screen_width)
    y = min(max(y, 0), screen_height)

    # get the actual x,y coordinates
    x_actual, y_actual = Y_test[i][0] * screen_width, Y_test[i][1] * screen_height

    # plot the predicted x,y coordinates
    cv2.circle(canvas, (int(x), int(y)), 10, (0, 0, 255), -1)
    # plot the actual x,y coordinates
    cv2.circle(canvas, (int(x_actual), int(y_actual)), 10, (0, 255, 0), -1 )
    # show the canvas
    cv2.imshow('Gaze Tracking on Canvas', canvas)
    cv2.waitKey(0)
    # # show the image 
    # cv2.imshow('image', X_test[i])
    # cv2.waitKey(0)
    # # clear the canvas
    
    canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)

cv2.waitKey(0)
cv2.destroyAllWindows()